In [1]:

import pandas as pd
import numpy as np
from pandas import DataFrame, Series

import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
data = pd.read_csv('NF_scenario_new.csv', encoding='cp949') #새로운 시나리오 csv 불러오기

In [3]:
# 시나리오 부여

# 어린이(13세 미만), 청소년 및 성인 따로 분류
data_child = data[data.연령 < 13]
data_other_work = data[(data.연령 > 12)&(data.work_daily > 0)|(data.school_daily > 0)]
data_other_home = data[(data.연령 > 12)&(data.work_daily == 0)&(data.school_daily == 0)]


# 학교 시나리오 분배를 위한 분배비율 default, 사용자가 수정할 경우 그 입력값을 prob로 받아옴
prob = {'school_1': .45,
        'school_2': .05,
        'school_3': .45,
        'school_4': .05} 

# 어린이의 학교 시나리오 분배, 청소년 및 성인의 other (직장 또는 학교) 시나리오 분배, 직장을 다니지 않는 성인의 시나리오 0 분배

data_child.loc[:,'other_sn'] = np.random.choice(list(prob.keys()), size=len(data_child), replace = True, p=list(prob.values()))
data_other_work.loc[:,'other_sn'] = 'work_1'
data_other_home.loc[:, 'other_sn'] = 0

data_1 = pd.concat([data_child, data_other_work, data_other_home], ignore_index=True)
data_1 = data_1.sort_values(by=['가구일련번호', '가구원일련번호'])

# 가정 시나리오 분배를 위한 분배비율 default, 사용자가 수정할 경우 그 입력값을 prob_1로 받아옴
prob_1 = {'home_1': .25,
          'home_2': .25,
          'home_3': .25,
          'home_4': .25} 


# 가구별 home_i 시나리오 분배
data_2 = data_1.groupby(['가구일련번호'], as_index=False).size()
data_2.loc[:, 'home_sn'] = np.random.choice(list(prob_1.keys()), size=len(data_2), replace = True, p=list(prob_1.values()))
data_2 = data_2.drop(columns='size')

# 가구별 home 시나리오를 개인별 데이터(data_1)과 합치고 data_3로 통합
data_3 = pd.merge(data_1, data_2, left_on='가구일련번호', right_on='가구일련번호', how='left')

In [4]:
#활동공간별 실내공기 및 먼지에 대하여 dummy 데이터 생성
# 항후 방출모델 결과를 활용
scenario = ['school_1', 'school_2', 'school_3', 'school_4', 'home_1', 'home_2', 'home_3', 'home_4', 'work_1']
other_c_air = pd.DataFrame(np.random.random_sample(9), index = scenario, columns=['other_c_air'])
home_c_air = pd.DataFrame(np.random.random_sample(9), index = scenario, columns=['home_c_air'])
other_c_dust = pd.DataFrame(np.random.random_sample(9), index = scenario, columns=['other_c_dust'])
home_c_dust = pd.DataFrame(np.random.random_sample(9), index = scenario, columns=['home_c_dust'])

# 활동공간별 농도값을 data 테이블에 합침

data_4 = pd.merge(data_3, other_c_air, left_on='other_sn', right_on=other_c_air.index, how='left')
data_4 = pd.merge(data_4, home_c_air, left_on='home_sn', right_on=home_c_air.index, how='left')
data_4 = pd.merge(data_4, other_c_dust, left_on='other_sn', right_on=other_c_dust.index, how='left')
data_4 = pd.merge(data_4, home_c_dust, left_on='home_sn', right_on=home_c_dust.index, how='left')
data_4.info()
data_4 = data_4.fillna(0) # NAN이 있는 경우 노출량 계산결과가 NAN으로 나오므로, NAN을 0값으로 대체

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21582 entries, 0 to 21581
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0.1    21582 non-null  int64  
 1   Unnamed: 0      21582 non-null  int64  
 2   회차              21582 non-null  object 
 3   행정구역시도코드        21582 non-null  object 
 4   읍면부동부코드         21582 non-null  object 
 5   가구일련번호          21582 non-null  int64  
 6   가구원일련번호         21582 non-null  int64  
 7   대표가구원여부         21582 non-null  object 
 8   평토일구분코드         21582 non-null  object 
 9   성별코드            21582 non-null  object 
 10  연령              21582 non-null  int64  
 11  연령코드            21582 non-null  object 
 12  etc_daily       21582 non-null  int64  
 13  home_daily      21582 non-null  int64  
 14  work_daily      21582 non-null  int64  
 15  school_daily    21582 non-null  int64  
 16  home_weekend    21582 non-null  int64  
 17  bodyweight      21582 non-null 

In [5]:
np_data1 = data_4.to_numpy() # 쉬운 계산을 위하여 numpy array로 변경

In [6]:
# 노출계수 정의
h_c_air = np_data1[:,23]  #집 공기중 농도
o_c_air = np_data1[:,22]  #학교, 직장 공기 중 농도
h_c_dust = np_data1[:,25] #집 먼지 중 농도
o_c_dust = np_data1[:,24] #학교, 직장 먼지 중 농도

BW = np_data1[:,17] #체중
h_d_time_ratio = np_data1[:,13]/1440 #주중 집에 24시간 중 머무는 시간 비율
o_d_time_ratio = (np_data1[:,14] + np_data1[:,15])/1440 # 주중 학교, 직장에 24시간 중 머무는 시간 비율
h_w_time_ratio = np_data1[:,16]/1440 # 주말 집에 24시간 중 머무는 시간 비율

igR = np_data1[:,18] # 먼지 섭취량
iR = np_data1[:,19] # 호흡률


In [7]:

# air exposure = conc * IR * AR / BW,                 air conc = conc * AR
# dust exposure = conc * dust_IgR * AR / BW

exp_inh_air_home_daily = h_c_air * iR * h_d_time_ratio / BW # 주중 집에 머무는 시간에 대한 실내공기 노출량
exp_inh_air_other_daily = o_c_air * iR * o_d_time_ratio / BW # 주중 활동공간에 머무는 시간에 대한 실내공기 노출량
exp_inh_air_home_weekend = h_c_air * iR * h_w_time_ratio / BW # 주말 집에 머무는 시간에 대한 실내공기 노출량

exp_inh_dust_home_daily = h_c_dust * igR * h_d_time_ratio / BW # 주중 집에 머무는 시간에 대한 먼지 노출량
exp_inh_dust_other_daily = o_c_dust * igR * o_d_time_ratio / BW # 주중 활동공간에 머무는 시간에 대한 먼지 노출량
exp_inh_dust_home_weekend = h_c_dust * igR * h_w_time_ratio / BW # 주말 집에 머무는 시간에 대한 먼지 노출량


# air_conc = conc * AR : 노출량(mg/kg-day)이 아닌 노출농도 계산(mg/m3)
conc_inh_air_home_daily = h_c_air * h_d_time_ratio # 주중 집에 머무는 시간에 대한 실내공기 노출농도
conc_inh_air_other_daily = o_c_air * o_d_time_ratio # 주중 활동공간에 머무는 시간에 대한 실내공기 노출농도
conc_inh_air_home_weekend = h_c_air * h_w_time_ratio # 주말 집에 머무는 시간에 대한 실내공기 노출농도


#일주일 평균 노출량 >> (5* daily exposure(home+other) + 2*weekend exposure )/7
mean_exp_air = ( 5*(exp_inh_air_home_daily + exp_inh_air_other_daily) + (2*exp_inh_air_home_weekend))/7 # 일주일 실내공기 평균 노출량

# 일주일 평균 노출농도 >> (5* daily exposure(home+other) + 2*weekend exposure )/7
mean_conc_air = ( 5*(conc_inh_air_home_daily + conc_inh_air_other_daily) + (2*conc_inh_air_home_weekend))/7 # 일주일 실내공기 평균 노출농도


mean_exp_dust = ( 5*(exp_inh_dust_home_daily + exp_inh_dust_other_daily) + (2*exp_inh_dust_home_weekend))/7 # 일주일 평균 먼지 노출량

total_exp = mean_exp_air + mean_exp_dust # 먼지 + 실내공기 통합 노출량





In [12]:
df_mean_exp_air = pd.DataFrame(mean_exp_air, columns=['mean_exp_air'])
df_mean_conc_air = pd.DataFrame(mean_conc_air, columns=['mean_conc_air'])
df_mean_exp_dust = pd.DataFrame(mean_exp_dust, columns=['mean_exp_dust'])
df_total_exp = pd.DataFrame(total_exp, columns=['total_exp'])



In [41]:


data_5 = pd.concat([data_4, df_mean_exp_air, df_mean_conc_air, df_mean_exp_dust, df_total_exp], axis = 1)

In [50]:
import plotly.express as px


fig = px.histogram(data_frame = data_5, x='home_daily', histnorm='density')
fig.update_layout(xaxis_title="주중 집에서 머무는 시간(분) - 전체", yaxis_title="Count")

fig_2 = px.histogram(data_frame = data_5, x='school_daily', histnorm='density')
fig_2.update_layout(xaxis_title="주중 학교에서 머무는 시간(분) - 전체", yaxis_title="Count")
# schooler로만 한정



fig_3 = px.histogram(data_frame = data_5, x='work_daily', histnorm='density')
fig_3.update_layout(xaxis_title="주중 직장에서 머무는 시간(분) - 전체", yaxis_title="Count")
# worker로만 한정


fig_4 = px.histogram(data_frame = data_5, x='home_weekend', histnorm='density')
fig_4.update_layout(xaxis_title="주말 가정에서 머무는 시간(분) - 전체", yaxis_title="Count")

fig.show()
fig_2.show()
fig_3.show()
fig_4.show()